In [4]:
from main_library import *
# %reload_ext dotenv
# %dotenv
# %reload_ext autoreload
# %autoreload 2

In [5]:
extractDate = date_load()
proj = projects_load()
proj_id_signed = proj.project_id.unique()

prop = proposals_load()
proj = proj_add_cols(prop, proj)

stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
prop1 = proposals_status(prop, proj_id_signed, stage_p) 
call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# project data missing in proposals if call already in proposals then add this
proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
call_miss = list(set(call_miss)-set(call_to_integrate))
proj = proj.loc[~proj.callId.isin(call_miss)]

# merge proj + prop
print('### MERGED PROPOSALS/PROJECTS')
if len(proj1)==0:
    prop2=pd.concat([proj,prop1], ignore_index= True)
else:
    prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")
merged = copy.deepcopy(prop2)
merged = dates_year(merged)
merged = strings_v(merged)
merged = url_to_clean(merged)
merged.mask(merged=='', inplace=True)
merged = empty_str_to_none(merged)      
merged.rename(columns={
    'freekw':'free_keywords',
    'callDeadlineDate':'call_deadline', 
    'callId':'call_id', 
    'submissionDate':'submission_date',
    'startDate':'start_date',
    'endDate':'end_date', 
    'ecSignatureDate':'signature_date'}, inplace=True)

if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
    print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

merged = merged_panels(merged)
merged = merged_topics(merged)
merged = merged_actions(merged)

# calls list
calls = call(PATH_SOURCE+FRAMEWORK+'/')
print("\n### CALLS+MERGED")
if len(merged.loc[merged.call_id.isnull()])>0:
        print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
else:
    call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
    print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

calls = calls_to_check(calls, call_id)

projects = projects_complete_cleaned(merged, extractDate)

### LAST DATE of EXTRACTED DATA
[{'extraction_date': '2024-10-09', 'framework': 'HORIZON'}]

### LOADING PROJECTS data
- no new columns
- result -> dowloaded projects:17382, retained projects:17382, pb:0
- liste des colonnes conservées:
Index(['project_id', 'callId', 'callDeadlineDate', 'typeOfActionCode',
       'acronym', 'status_code', 'startDate', 'endDate', 'ecSignatureDate',
       'title', 'abstract', 'url', 'topicCode', 'duration', 'total_cost',
       'eu_reqrec_grant', 'number_involved', 'framework', 'lastUpdateDate',
       'totalGrant', 'nationalContribution', 'otherContribution', 'freekw',
       'stage'],
      dtype='object')

### LOADING PROPOSALS data
- no new columns
1- empty cols -> Attention ! vérifier les variables manquantes->[]
- result -> dowloaded proposals:92864, retained proposals:92864, pb:0

### ADD COLS TO PROJECTS FROM PROPOSALS
- liste des variables PROJ en plus: ['ecSignatureDate', 'status_code', 'totalGrant', 'endDate', 'nationalContribution', 'startDa

c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


- code panel existant pour d'autres programmes que erc/msca: données exportées

- size merged after add panels: 102295
### TOPICS
1 - topics -> 2448
2 - divisions -> 3172
erc : destination_code à null après traitement
['HORIZON-ERC-2021-VICECHAIRS-IBA' 'ERC-2025-NCPS-IBA'
 'HORIZON-ERC-2021-arXiv-IBA'
 'HORIZON-CL4-2024-SSA-SST-SD-IBA-to-be-deleted'
 'HORIZON-CL4-2024-SSA-SST-AE-IBA-to-be-deleted'
 'HORIZON-ERC-2023-VICECHAIRS-IBA' 'HORIZON-ERC-2022-VICECHAIRS-IBA'
 'HORIZON-CL4-2024-SSA-SST-SP-IBA-to-be-deleted']
MSCA : destination_code à null après traitement
['HORIZON-MSCA-2021-INCO-01-01' 'HORIZON-MSCA-2021-FRC-IBA'
 'HORIZON-MSCA-2022-ALUMNI-IBA' 'HORIZON-MSCA-2024-NCP-01-01'
 'HORIZON-MSCA-2023-FTP-01-01' 'HORIZON-MSCA-2022-Ukraine-ART195-IBA'
 'HORIZON-MSCA-2021-RR-01-01' 'HORIZON-MSCA-2024-RR-01-01'
 'HORIZON-MSCA-2023-BEL-IBA' 'HORIZON-MSCA-2024-UKRAINE-IBA'
 'HORIZON-MSCA-2024-INCO-01-01' 'HORIZON-MSCA-SNLS-2021-IBA'
 'HORIZON-MSCA-2021-NCP-01-01' 'HORIZON-MSCA-2024-ALUMNI-IB

c:\Users\zfriant\Documents\GitHub\pcri\step1_mainData\topics.py:171: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SESAR' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top.loc[mask, 'destination_code'] = i


size merged after add topics: 102295

### ACTIONS
1- call sans action:
                                      call_id
24693  HORIZON-HLTH-2022-DISEASE-06-two-stage

- size merged after add actions: 102295


### CALLS

### CALLS+MERGED
2 - CALL_ID de merged -> nb call+deadline: 530, nb call unique: 492 

### CALLS to CHECK
- CALLS de base calls with merge call_id ->
nb call+deadline: 537, nb call unique: 492 
2- si calls pas dans call_id :
                             call_id call_deadline
231  HORIZON-EIC-2023-BOOSTER-IBA-01    2023-07-05
248  HORIZON-EIC-2024-ACCELERATOR-02    2024-10-03
251  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-09-04
252  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-11-14
467       HORIZON-MSCA-2021-SNLS-IBA    2021-07-27
505    HORIZON-WIDERA-2022-ACCESS-07    2022-04-20
522    HORIZON-WIDERA-2023-ACCESS-06    2024-09-26
3- calls with multi deadline
                                     call_id call_deadline   call_budget
17                              ERC-2023-POC    

In [6]:
##### PARTICIPATIONS
part = participants_load(proj)
# conserve uniquement les projets présents dans proposals et applicants
part = part.loc[part.project_id.isin(projects.project_id.unique())]
print(f"- size part hors proj manquant: {len(part)}")
part = part_role_type(part)
part = erc_role(part, projects)

#### APPLICANTS
app = applicants_load(prop)
# conserve uniquement les projets présents dans proposals et applicants
app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
print(f"- size app1 hors proj exclus: {len(app1)}")

app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
tmp = part[part.project_id.isin(app_missing_pid)]
app1 = part_miss_app(tmp, app1)

#redressement accelerator
acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
print(f"size acc_folio: {len(acc_folio)}")
acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
       .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
       .drop(columns='PROPOSAL_NBR'))
print(f"size acc: {len(acc)}")
app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

app1 = app_role_type(app1)
app1 = erc_role(app1, projects)

del app

####
# verification Etat des participations
part = check_multiP_by_proj(part)
app1 = check_multiA_by_proj(app1)



### LOADING PARTICIPANTS data
- no new columns
- subv_net avant traitement: 44,801,576,700.0
- suppression de 1434,
modalités: partnerRemovalStatus
TERMINATED              1343
NOT_ACCEDED               91
Name: count, dtype: int64
4- projets dans participants et pas dans projects
#FCT bugs_excel
#FCT gps_col
- result -> dowloaded:105091, retained part:103667, pb:1424, somme euContribution:44,757,770,154.3, somme netEuContribution:44,758,956,133.3
- size part hors proj manquant: 100423
### Participants ROLE
- size part after role: 100423
### ERC ROLE
- size after erc_role: 100423

### LOADING APPLICANTS data
- size app au chargement: 473219
- no new columns
- var with null: ['shortName', 'countryCode', 'nutsCode', 'naceCode', 'budget', 'requestedGrant']
#FCT gps_col
- result -> dowloaded:473219, retained app:473219, pb:0, , somme requestedGrant:225,526,448,130.6
- size app1 hors proj exclus: 463510

### add to APPLICANT from PARTICIPANT
1- ATTENTION ! vont être ajoutés les participant

In [7]:
lien = merged_partApp(app1, part)
lien = nuts_lien(app1, part, lien)


### create LIEN


app2: 463523 part2:100423
- applicant uniquement lien1: 366542 reste à croiser -> app2: 96981
jointure parfaite -> lien2: 56241
reste à croiser -> app3: 40740 part3: 44182
jointure sans ordernumber -> lien3: 36179
reste à croiser -> app4: 4574 part4: 8013
jointure sans participant_pic -> lien4: 673
reste à croiser -> app5: 3901 part5: 7340
jointure seulement avec generalpic -> lien5: 628
reste à croiser -> app6: 3273 part6: 6712
jointure avec seulement participant_pic -> lien6: 0
- size lien: 470876

### LOADING DEPARTMENT
- size pp_app: 353303
- size pp_part: 89566

### NUTS avec LIEN
- size nuts_a: 463523, size nuts_p: 100423
size lien after add nuts: 470876, sans code_nuts: 0
- size lien: 470876


In [11]:
def entities_missing_country(df):

    if any(df['countryCode'].isnull()):
        print(f"2 - ATTENTION missing {len(df['countryCode'].isnull())} countryCode")
        df.loc[df['countryCode'].isnull(), 'countryCode'] = df.loc[df['countryCode'].isnull(), 'countryCode_y']
    if len(df.loc[df.countryCode.isnull()])>0:
        print(f"3 - ATTENTION ! missing again countryCode {df.loc[df.countryCode.isnull(), ['generalPic']].drop_duplicates()}")
    else:
        print(f'4 - RESOLU -> sans country\n- size entities with cc: {len(df)}')
    return df


def entities_load(lien):

    entities = unzip_zip(ZIPNAME, f"{PATH_SOURCE}{FRAMEWORK}/", "legalEntities.json", 'utf8')
    entities = pd.DataFrame(entities)
    print(f"- first size entities: {len(entities)}")
    entities = gps_col(entities)

    entities = entities.loc[~entities.generalPic.isnull()]

    c = ['pic', 'generalPic']
    entities[c] = entities[c].map(num_to_string)
    print(f"- size entities {len(entities)}")


    ap=(lien.loc[lien.inProposal==True, ['generalPic', 'proposal_participant_pic', 'proposal_countryCode']]
        .drop_duplicates()
        .rename(columns={'proposal_participant_pic':'pic', 'proposal_countryCode':'countryCode'}))
    pp=(lien.loc[lien.inProject==True, ['generalPic', 'participant_pic', 'countryCode']]
        .drop_duplicates()
        .rename(columns={'participant_pic':'pic'}))
    tmp=pd.concat([ap, pp], ignore_index=True).drop_duplicates()
    print(f"- size lien ap+pp+cc (tmp): {len(tmp)}")
    entities=tmp.merge(entities, how='left', on=['generalPic', 'pic']).rename(columns={'countryCode_x': 'countryCode'})
    print(f"- size tmp+entities: {len(entities)}")

    if len(tmp)!=len(entities):
        print(f"1 - ATTENTION missing generalPic into entities")

    entities = entities_missing_country(entities)

    # selection des obs de entities liées aux participants/applicants
    # entities = entities.loc[entities.generalPic.isin(lien.generalPic.unique())]
    print(f"- END size entities: {len(entities)}")  

    pic_no_entities = list(set(lien.generalPic.unique()) - set(entities.generalPic.unique()))
    if len(pic_no_entities) >0:
        print(f"- pic lien not in entities: {len(pic_no_entities)}")
    else:
        print("- Tous les pics de lien sont dans entities")

    return entities

entities = entities_load(lien)




- first size entities: 283425
#FCT gps_col
- size entities 283422
- size lien ap+pp+cc (tmp): 79350
- size tmp+entities: 79350
4 - RESOLU -> sans country
- size entities with cc: 79350
- END size entities: 79350
- Tous les pics de lien sont dans entities


In [12]:
entities[entities.generalPic=='877256167']

,generalPic,pic,countryCode,legalName,businessName,generalState,street,postalCode,postalBox,city,...,isHigherEducation,legalEntityTypeCode,legalEntityType,legalType,legalStatus,vat,legalRegNumber,naceCode,lastUpdateDate,gps_loc
49249,877256167,900310545,NL,EURACTIV MEDIA NETWORK,None,SUSPENDED,KATWILGWEG 2,2050,000,ANTWERP,...,False,PRC,Private for-profit entities (excluding Higher ...,PRIVATE,LEGAL_PERSON,BE01003987216,1003987216,None,2024-10-06,"51.17417,4.45933"
78009,877256167,900310545,BE,EURACTIV MEDIA NETWORK,None,SUSPENDED,KATWILGWEG 2,2050,000,ANTWERP,...,False,PRC,Private for-profit entities (excluding Higher ...,PRIVATE,LEGAL_PERSON,BE01003987216,1003987216,None,2024-10-06,"51.17417,4.45933"
78111,877256167,877256167,BE,EURACTIV MEDIA BV,None,VALIDATED,BOULEVARD CHARLEMAGNE 1 BOITE 1,1000,.,BRUSSELS,...,False,PRC,Private for-profit entities (excluding Higher ...,PRIVATE,LEGAL_PERSON,BE0457860784,457860784,None,2024-10-06,"50.87776,4.32609"


In [ ]:
def entities_cleaning(df):
    # contrôle nombre d'obs avec les pic coutry et state
    PicState=df[['generalPic', 'generalState', 'countryCode']]
    n_state=PicState.groupby(['generalPic',  'countryCode']).filter(lambda x: x['generalState'].count() > 1.)

    if (len(n_state)>0):
        print(f"1 - ++state pour un pic/country; régler ci-dessous {len(n_state)}")
        gen_state=['VALIDATED', 'DECLARED', 'DEPRECATED', 'SLEEPING', 'SUSPENDED', 'BLOCKED']

        if len(df.generalState.unique()) > len(gen_state):
            print(f"2 - Attention ! un generalState nouveau dans entities -> {set(df.generalState.unique())-set(gen_state)}")
        else:
            df=df.groupby(['generalPic', 'countryCode']).apply(lambda x: x.sort_values('generalState', key=lambda col: pd.Categorical(col, categories=gen_state, ordered=True)), include_groups=True).reset_index(drop=True)
            df=df.groupby(['generalPic', 'countryCode']).head(1)
            print(f"3 - size entities: {len(df)}")
    return df

entities_tmp = entities_cleaning(entities)

1 - ++state pour un pic/country; régler ci-dessous 2631
3 - size entities: 79350


C:\Users\zfriant\AppData\Local\Temp\ipykernel_19724\559982292.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby(['generalPic', 'countryCode']).apply(lambda x: x.sort_values('generalState', key=lambda col: pd.Categorical(col, categories=gen_state, ordered=True))).reset_index(drop=True)


In [15]:
entities_tmp=entities_tmp.groupby(['generalPic', 'countryCode']).head(1)
print(f"3 - size entities: {len(entities_tmp)}")
entities_tmp[entities_tmp.generalPic=='877256167']

3 - size entities: 77991


,generalPic,pic,countryCode,legalName,businessName,generalState,street,postalCode,postalBox,city,...,isHigherEducation,legalEntityTypeCode,legalEntityType,legalType,legalStatus,vat,legalRegNumber,naceCode,lastUpdateDate,gps_loc
215,877256167,877256167,BE,EURACTIV MEDIA BV,None,VALIDATED,BOULEVARD CHARLEMAGNE 1 BOITE 1,1000,.,BRUSSELS,...,False,PRC,Private for-profit entities (excluding Higher ...,PRIVATE,LEGAL_PERSON,BE0457860784,457860784,None,2024-10-06,"50.87776,4.32609"
217,877256167,900310545,NL,EURACTIV MEDIA NETWORK,None,SUSPENDED,KATWILGWEG 2,2050,000,ANTWERP,...,False,PRC,Private for-profit entities (excluding Higher ...,PRIVATE,LEGAL_PERSON,BE01003987216,1003987216,None,2024-10-06,"51.17417,4.45933"


In [ ]:
print(f"- size entities_single:{len(entities_tmp)}\n{entities_tmp.generalState.value_counts()}")
if (entities_tmp.generalPic.nunique())==(lien.generalPic.nunique()):
    print(f"\n1 - nombre de pics OK")
#si pas le m^me nombre de pics entre lien et entities
elif len(set(lien.generalPic.unique()))>len(set(entities.generalPic.unique())):
    pic_lien=list(set(lien.generalPic.unique()) - set(entities.generalPic.unique()))
    print(f"\n2 - pic_lien absent de entities_single {pic_lien}")

tmp=entities_tmp.groupby(['generalPic', 'countryCode']).filter(lambda x: x['generalPic'].count() > 1.)
if not tmp.empty:
    print(f"1 - ATTENTION doublon generalPic revoir code ci-dessous si besoin")

- size entities_single:77991
generalState
VALIDATED     40342
DECLARED      35818
SLEEPING       1524
DEPRECATED      278
SUSPENDED        17
BLOCKED          12
Name: count, dtype: int64
1 - nombre de pics OK
